In [2]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tahtan
# Import Sionna
import sys
sys.path.append('../')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig, TBConfig, PUSCHPilotPattern, TBEncoder
from sionna.nr.utils import generate_prng_seq
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import * 
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f, BinarySource
from sionna.ofdm import KBestDetector, LinearDetector, MaximumLikelihoodDetector, ResourceGrid, ResourceGridMapper, OFDMModulator
from sionna.mimo import StreamManagement


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
import json
from tqdm.notebook import tqdm
import itertools

In [4]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self):
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=0,
                cyclic_prefix="normal",
                subcarrier_spacing=30,
                n_size_grid=273,
                n_start_grid=0,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=1,
                length=1,
                additional_position=1,
                dmrs_port_set=[0],
                n_id=0,
                n_scid=0,
                num_cdm_groups_without_data=2,
                type_a_position=2
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=0,
                mcs_table=1,
                mcs_index=9
            ),
            mapping_type='A',
            n_size_bwp=273,
            n_start_bwp=0,
            num_layers=1,
            num_antenna_ports=1,
            precoding='non-codebook',
            tpmi=0,
            transform_precoding=False,
            n_rnti=2008,
            symbol_allocation=[0,14]
        )

pusch_config_0 = MyPUSCHConfig()
pusch_config_0.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 0
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 3276, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (3276, 14)
dmrs_symbol_indices : [2, 11]
frequency_hopping : neither
l : [2, 11]
l_0 : 2
l_bar : [2, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (819,)
n_rnti : 2008
n_size_bwp : 273
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 78624
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 273
num_subcarriers : 3276
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [0, 14]
tb_size : 52224
tpmi : 0
transform_precoding : False

PUSCH 

In [5]:
num_tx = 1
num_rx = 1
num_rx_ant = 8
num_tx_ant = 1
carrier_frequency = 2.55e-9

# Configure antenna arrays
ue_antenna = Antenna(polarization="single",
                polarization_type="V",
                antenna_pattern="38.901",
                carrier_frequency=carrier_frequency)

gnb_array = AntennaArray(num_rows=1,
                        num_cols=num_rx_ant//2,
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=carrier_frequency)

channel_model_0 = CDL("C", 150e-9, carrier_frequency, ue_antenna, gnb_array, "uplink", min_speed=10)

In [6]:
def setCfgReg(pusch_config, dir):
    filename = os.path.join(dir, 'cfgReq.cfg')

    with open(filename, 'w') as file:
        file.write('<?xml version="1.0"?>\n\n')
        file.write('<TestConfig>\n')
        
        file.write(f'\t<numSlots>{pusch_config.carrier.num_slots_per_frame}</numSlots>\n')

        NRxAnt = 8
        for rxIdx in range(NRxAnt):
            file.write(f'\t<uliq_car0_ant{rxIdx}>rx_ant_{rxIdx}.bin</uliq_car0_ant{rxIdx}>\n')

        file.write('\t<ul_ref_out>ref.txt</ul_ref_out>\n')
        FrameNumber = 0
        file.write(f'\t<start_frame_number>{FrameNumber}</start_frame_number>\n')
        file.write(f'\t<start_slot_number>{0}</start_slot_number>\n')
        file.write('</TestConfig>\n\n')

        file.write('<ConfigReq>\n')
        file.write('\t<nCarrierIdx>0</nCarrierIdx>\n')
        file.write(f'\t<nDMRSTypeAPos>{pusch_config.dmrs.type_a_position}</nDMRSTypeAPos>\n')
        file.write(f'\t<nPhyCellId>{pusch_config.carrier.n_cell_id}</nPhyCellId>\n')
        
        Bandwidth = 100
        file.write(f'\t<nDLBandwidth>{Bandwidth}</nDLBandwidth>\n')  # Hardcoded same as UL
        file.write(f'\t<nULBandwidth>{Bandwidth}</nULBandwidth>\n')  # Hardcoded same as UL

        Nfft = 2**np.ceil(np.log2(pusch_config.carrier.n_size_grid * 12 / 0.85))
        Nfft = max(128, int(Nfft)) 
        file.write(f'\t<nDLFftSize>{Nfft}</nDLFftSize>\n')
        file.write(f'\t<nULFftSize>{Nfft}</nULFftSize>\n')

        file.write(f'\t<nNrOfTxAnt>{NRxAnt}</nNrOfTxAnt>\n')
        file.write(f'\t<nNrOfRxAnt>{NRxAnt}</nNrOfRxAnt>\n')
        file.write(f'\t<nCarrierAggregationLevel>0</nCarrierAggregationLevel>\n')
        file.write(f'\t<nFrameDuplexType>1</nFrameDuplexType>\n')
        file.write(f'\t<nSubcCommon>{pusch_config.carrier.mu}</nSubcCommon>\n')
        file.write(f'\t<nTddPeriod>20</nTddPeriod>\n')

        slotSetInFrame = [4,5,14,15]
        for slotIdx in range(pusch_config.carrier.num_slots_per_frame):
            if slotIdx in slotSetInFrame:
                file.write(f'\t<sSlotConfig{slotIdx}>1,1,1,1,1,1,1,1,1,1,1,1,1,1</sSlotConfig{slotIdx}>\n')
            else:
                file.write(f'\t<sSlotConfig{slotIdx}>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig{slotIdx}>\n')

        cpType = int(pusch_config.carrier.cyclic_prefix!='normal')
        file.write(f'\t<nCyclicPrefix>{cpType}</nCyclicPrefix>\n')
        file.write('</ConfigReq>\n\n')

        file.write('<RxConfig>\n')
        for slotIdx in range(pusch_config.carrier.num_slots_per_frame):
            if slotIdx in slotSetInFrame:
                file.write(f'\t<SlotNum{slotIdx}>slot{slotIdx}.cfg</SlotNum{slotIdx}>\n')
            else:
                file.write(f'\t<SlotNum{slotIdx}>null.cfg</SlotNum{slotIdx}>\n')

        file.write('</RxConfig>\n')

setCfgReg(pusch_config_0, '../ABC')

In [7]:
def setUlCfgReq(pusch_config, dir):
    harqIdx = -1
    slotSetInFrame = [4,5,14,15]
    for slotIdx in slotSetInFrame:
        harqIdx += 1
        filename = os.path.join(dir, f"slot{slotIdx}.cfg")

        with open(filename, 'w') as file:
            file.write('<?xml version="1.0"?>\n\n')
            file.write('<Ul_Config_Req>\n\n')
            file.write('\t<UlConfigReqL1L2Header>\n')
            FrameNumber = 0
            file.write(f'\t\t<nSFN>{FrameNumber}</nSFN>\n')
            file.write(f'\t\t<nSlot>{slotIdx}</nSlot>\n')
            nUe=1
            nMuMimoGroup=1
            file.write(f'\t\t<nPDU>{nUe}</nPDU>\n')  # TBD
            file.write(f'\t\t<nGroup>{nMuMimoGroup}</nGroup>\n')  # TBD
            file.write(f'\t\t<nUlsch>{nUe}</nUlsch>\n')  # TBD
            file.write(f'\t\t<nUlcch>0</nUlcch>\n')
            file.write(f'\t\t<nRachPresent>0</nRachPresent>\n')
            file.write('\t</UlConfigReqL1L2Header>\n\n')

            # for ueIdx in range(nUe):
            ueIdx = 0
            file.write(f'\t<UL_SCH_PDU{ueIdx}>\n')
            file.write(f'\t\t<nRNTI>{pusch_config.n_rnti}</nRNTI>\n')
            file.write(f'\t\t<nUEId>{ueIdx}</nUEId>\n')
            file.write(f'\t\t<nBWPSize>{pusch_config.carrier.n_size_grid}</nBWPSize>\n')
            file.write(f'\t\t<nBWPStart>{pusch_config.carrier.n_start_grid}</nBWPStart>\n')
            file.write(f'\t\t<nSubcSpacing>{pusch_config.carrier.mu}</nSubcSpacing>\n')

            cpType = int(pusch_config.carrier.cyclic_prefix!='normal')
            file.write(f'\t\t<nCpType>{cpType}</nCpType>\n')
            file.write(f'\t\t<nULType>0</nULType>\n')
                
            file.write(f'\t\t<nMcsTable>{pusch_config.tb.mcs_table - 1}</nMcsTable>\n')
            file.write(f'\t\t<nMCS>{pusch_config.tb.mcs_index}</nMCS>\n')
            file.write(f'\t\t<nTransPrecode>{int(pusch_config.transform_precoding)}</nTransPrecode>\n')
            precoding = int(pusch_config.precoding != 'non-codebook')
            file.write(f'\t\t<nTransmissionScheme>{precoding}</nTransmissionScheme>\n')
            file.write(f'\t\t<nNrOfLayers>{pusch_config.num_layers}</nNrOfLayers>\n')

            file.write(f'\t\t<nPortIndex{0}>{0}</nPortIndex{0}>\n')

            file.write(f'\t\t<nNid>{pusch_config.tb.n_id}</nNid>\n')
            file.write(f'\t\t<nSCID>{pusch_config.dmrs.n_scid}</nSCID>\n')
            file.write(f'\t\t<nNIDnSCID>{pusch_config.dmrs.n_id[0]}</nNIDnSCID>\n')

            NRxAnt = 8
            file.write(f'\t\t<nNrOfAntennaPorts>{NRxAnt}</nNrOfAntennaPorts>\n')

            file.write(f'\t\t<nVRBtoPRB>0</nVRBtoPRB>\n')
            file.write(f'\t\t<nPMI>{pusch_config.tpmi}</nPMI>\n')
            file.write(f'\t\t<nStartSymbolIndex>{pusch_config.symbol_allocation[0]}</nStartSymbolIndex>\n')
            file.write(f'\t\t<nNrOfSymbols>{pusch_config.symbol_allocation[1]}</nNrOfSymbols>\n')
            
            ra_type = 1
            file.write(f'\t\t<nResourceAllocType>{ra_type}</nResourceAllocType>\n')
            file.write(f'\t\t<nRBStart>{pusch_config.n_start_bwp }</nRBStart>\n')
            file.write(f'\t\t<nRBSize>{pusch_config.n_size_bwp }</nRBSize>\n')
            file.write(f'\t\t<nTBSize>{pusch_config.tb_size // 8}</nTBSize>\n')

            rv = 0
            file.write(f'\t\t<nRV>{rv}</nRV>\n')
            file.write(f'\t\t<nHARQID>{harqIdx}</nHARQID>\n')
            ndi = 1
            file.write(f'\t\t<nNDI>{ndi}</nNDI>\n')

            mapping_type = int(pusch_config.mapping_type != 'A')
            file.write(f'\t\t<nMappingType>{mapping_type}</nMappingType>\n')
            file.write(f'\t\t<nDMRSConfigType>{pusch_config.dmrs.config_type - 1}</nDMRSConfigType>\n')
            file.write(f'\t\t<nNrOfCDMs>{pusch_config.dmrs.num_cdm_groups_without_data}</nNrOfCDMs>\n')
            file.write(f'\t\t<nNrOfDMRSSymbols>{pusch_config.dmrs.length }</nNrOfDMRSSymbols>\n')
            file.write(f'\t\t<nDMRSAddPos>{pusch_config.dmrs.additional_position }</nDMRSAddPos>\n')
            file.write(f'\t\t<nPTRSPresent>0</nPTRSPresent>\n')

            OAck = 0
            file.write(f'\t\t<nAck>{OAck}</nAck>\n')
            ScalingFactor = 0
            file.write(f'\t\t<nAlphaScaling>{ScalingFactor}</nAlphaScaling>\n')
            IHarqAckOffset = 0
            file.write(f'\t\t<nBetaOffsetACKIndex>{IHarqAckOffset}</nBetaOffsetACKIndex>\n')
            OCsi1 = 0
            file.write(f'\t\t<nCsiPart1>{OCsi1}</nCsiPart1>\n')
            ICsi1Offset = 0
            file.write(f'\t\t<nBetaOffsetCsiPart1Index>{ICsi1Offset}</nBetaOffsetCsiPart1Index>\n')
            OCsi2 = 0
            file.write(f'\t\t<nCsiPart2>{OCsi2}</nCsiPart2>\n')
            ICsi2Offset = 0
            file.write(f'\t\t<nBetaOffsetCsiPart2Index>{ICsi2Offset}</nBetaOffsetCsiPart2Index>\n')
           
            TpPi2Bpsk = 0
            file.write(f'\t\t<nTpPi2BPSK>{TpPi2Bpsk}</nTpPi2BPSK>\n')
            NRsId = 0
            file.write(f'\t\t<nTPPuschID>{NRsId}</nTPPuschID>\n')
            for rxAntPortIdx in range(NRxAnt):
                file.write(f'\t\t<nRxRUIdx{rxAntPortIdx}>{rxAntPortIdx}</nRxRUIdx{rxAntPortIdx}>\n')

            file.write(f'\t</UL_SCH_PDU{ueIdx}>\n\n')
            
            groupIdx = 0
            file.write(f'\t<PUSCH_GROUP_INFO{groupIdx}>\n')
            file.write(f'\t\t<nUE>{nUe}</nUE>\n')
            file.write(f'\t\t<nPduIdx{ueIdx}>{ueIdx}</nPduIdx{ueIdx}>\n')
            file.write(f'\t</PUSCH_GROUP_INFO{groupIdx}>\n\n')

            file.write('</Ul_Config_Req>\n')

setUlCfgReq(pusch_config_0, '../ABC')

In [8]:
def set_null(dir):
    filename = os.path.join(dir, 'null.cfg')
    with open(filename, 'w') as file:
        file.write('<?xml version="1.0"?>\n\n')
        file.write('<Ul_Config_Req>\n')
        file.write('\t<UlConfigReqL1L2Header>\n')
        file.write('\t\t<nSFN>0</nSFN>\n')
        file.write('\t\t<nSlot>0</nSlot>\n')
        file.write('\t\t<nPDU>0</nPDU>\n')
        file.write('\t\t<nGroup>0</nGroup>\n')
        file.write('\t\t<nUlsch>0</nUlsch>\n')
        file.write('\t\t<nUlcch>0</nUlcch>\n')
        file.write('\t\t<nRachPresent>0</nRachPresent>\n')
        file.write('\t</UlConfigReqL1L2Header>\n')
        file.write('</Ul_Config_Req>\n')
set_null('../ABC')


In [9]:
# source_bits = generate_prng_seq(pusch_config.tb_size, 1234).reshape(1,1,-1)
# tx_IQ = PUSCHTransmitter(pusch_config, return_bits=False)(source_bits)

In [10]:
b_slot4 = generate_prng_seq(pusch_config_0.tb_size, 4).reshape(1,1,-1)
pusch_config_0.carrier.slot_number = 4
transmitter_slot4 = PUSCHTransmitter(pusch_config_0, return_bits=False)
x_slot4 = transmitter_slot4(b_slot4)

In [11]:
x_slot4

<tf.Tensor: shape=(1, 1, 1, 14, 3276), dtype=complex64, numpy=
array([[[[[-0.70710677-0.70710677j, -0.70710677-0.70710677j,
           -0.70710677-0.70710677j, ..., -0.70710677+0.70710677j,
           -0.70710677+0.70710677j, -0.70710677-0.70710677j],
          [ 0.70710677-0.70710677j, -0.70710677-0.70710677j,
           -0.70710677-0.70710677j, ..., -0.70710677+0.70710677j,
            0.70710677+0.70710677j,  0.70710677+0.70710677j],
          [-1.        -1.j        ,  0.        +0.j        ,
           -1.        -1.j        , ...,  0.        +0.j        ,
           -1.        -1.j        ,  0.        +0.j        ],
          ...,
          [ 1.        +1.j        ,  0.        +0.j        ,
            1.        +1.j        , ...,  0.        +0.j        ,
            1.        -1.j        ,  0.        +0.j        ],
          [ 0.70710677+0.70710677j,  0.70710677-0.70710677j,
           -0.70710677+0.70710677j, ...,  0.70710677+0.70710677j,
            0.70710677-0.70710677j,  0.

In [12]:
channel = OFDMChannel(channel_model_0,
                    transmitter_slot4.resource_grid,
                    return_channel=True)
y_slot4,_ = channel([x_slot4, 0.1])

In [13]:
b_slot5 = generate_prng_seq(pusch_config_0.tb_size, 5).reshape(1,1,-1)
pusch_config_0.carrier.slot_number = 5
x_slot5 = PUSCHTransmitter(pusch_config_0, return_bits=False)(b_slot5)

In [14]:
channel = OFDMChannel(channel_model_0,
                    transmitter_slot4.resource_grid,
                    return_channel=True)
y_slot5,_ = channel([x_slot5, 0.1])

In [15]:
b_slot14 = generate_prng_seq(pusch_config_0.tb_size, 14).reshape(1,1,-1)
pusch_config_0.carrier.slot_number = 14
x_slot14 = PUSCHTransmitter(pusch_config_0, return_bits=False)(b_slot14)
channel = OFDMChannel(channel_model_0,
                    transmitter_slot4.resource_grid,
                    return_channel=True)
y_slot14,_ = channel([x_slot14, 0.1])

In [16]:
b_slot15 = generate_prng_seq(pusch_config_0.tb_size, 15).reshape(1,1,-1)
pusch_config_0.carrier.slot_number = 15
x_slot15 = PUSCHTransmitter(pusch_config_0, return_bits=False)(b_slot15)
channel = OFDMChannel(channel_model_0,
                    transmitter_slot4.resource_grid,
                    return_channel=True)
y_slot15,_ = channel([x_slot15, 0.1])

In [17]:
y_slot_0 = tf.zeros_like(y_slot15)
y_slot_0

<tf.Tensor: shape=(1, 1, 8, 14, 3276), dtype=complex64, numpy=
array([[[[[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

         [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

         [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.

In [18]:
rxSigFreq = tf.squeeze(tf.stack([
                        y_slot_0, #0
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        tf.repeat(x_slot4, 8, axis=2),
                        tf.repeat(x_slot5, 8, axis=2),
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0, #10
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        tf.repeat(x_slot14, 8, axis=2),
                        tf.repeat(x_slot15, 8, axis=2),
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0],
                        axis=3))

rxSigFreq

<tf.Tensor: shape=(8, 20, 14, 3276), dtype=complex64, numpy=
array([[[[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0

In [19]:
rxSigFreq = tf.squeeze(tf.stack([
                        y_slot_0, #0
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot4,
                        y_slot5,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0, #10
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot14,
                        y_slot15,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0,
                        y_slot_0],
                        axis=3))

rxSigFreq

<tf.Tensor: shape=(8, 20, 14, 3276), dtype=complex64, numpy=
array([[[[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0

In [20]:
nRxAnt = 8
def setRxData(rxSigFreq, dir):
    rxSigFreq = tf.reshape(rxSigFreq, -1)
    rxSigFreq = tf.reshape(tf.stack((tf.math.real(rxSigFreq), tf.math.imag(rxSigFreq)), axis=-1), -1)
    
    rxSigFreq = rxSigFreq/tf.math.reduce_max(tf.abs(rxSigFreq))

    rxSigFreq = tf.cast(tf.round(rxSigFreq*2**13), tf.int16)
    rxSigFreq = tf.reshape(rxSigFreq,[nRxAnt,-1])
    
    for rxIdx in range(nRxAnt):
        file_path = os.path.join(dir, f'rx_ant_{rxIdx}.bin')
        with open(file_path, 'wb') as file:
            file.write(rxSigFreq[rxIdx])

setRxData(rxSigFreq, '../ABC')

In [21]:
slots = [
    None, # 0
    None,
    None,
    None,
    b_slot4,
    b_slot5,
    None,
    None,
    None,
    None,
    None, # 10
    None,
    None,
    None,
    b_slot14,
    b_slot15,
    None,
    None,
    None,
    None,
]

In [22]:
def setRef(pusch_config, slots, dir):
    filename = os.path.join(dir, 'ref.txt')
    slotSetInFrame = [4,5,14,15]
    with open(filename, 'w') as file:
        file.write(f'##----------------------------------------------------------------------------\n')
        for slotIdx in range(20):
            if slotIdx in slotSetInFrame:
                sfn = 0
                slot = slotIdx%20
                tbSize = pusch_config.tb_size//8
                file.write(f'#type[PUSCH] fn[{sfn}] slot[{slot}] sym[0] carrier[0] chanId[0] len[{tbSize}]\n')
                file.write(f'\t  #ta[0] cqi[0.0] stat[1]\n')
                file.write(f'\t  #data[\n')
                nRow = tbSize//64
                nRest = tbSize%64
                payload = b2b(slots[slotIdx])
                print(payload.shape, nRow, nRest)
                for rowIdx in range(nRow):
                    file.write('\t        ')
                    for colIdx in range(64):
                        file.write(f'{payload[64*rowIdx + colIdx]:3d}, ')
                    file.write('\n')
                file.write('\t        ')
                for colIdx in range(nRest-1):
                    file.write(f'{payload[64*nRow + colIdx]:3d}, ')
                file.write(f'{payload[-1]:3d}\n')
                file.write(f'\t       ]\n')
                file.write(f'------------------------------------------------------------------------------\n')
            else: 
                file.write(f'##----------------------------------------------------------------------------\n')

setRef(pusch_config_0, slots, '../ABC')

(6528,) 102 0
(6528,) 102 0
(6528,) 102 0
(6528,) 102 0
